In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('myproj').getOrCreate()

In [3]:
data = spark.read.csv('/FileStore/tables/titanic.csv',inferSchema=True,header=True)

In [4]:
data.printSchema()

In [5]:
data.columns

In [6]:
data.show()

In [7]:
my_cols = data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [8]:
my_final_data = my_cols.na.drop()

In [9]:
my_final_data.show()

In [10]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [11]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [12]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [13]:
assembler = VectorAssembler(inputCols=['Pclass',
 'SexVec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'EmbarkVec'],outputCol='features')

In [14]:
from pyspark.ml.classification import LogisticRegression


In [15]:
from pyspark.ml import Pipeline

In [16]:
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7,.3])


In [17]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [18]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                           gender_encoder,embark_encoder,
                           assembler,log_reg_titanic])

In [19]:
fit_model = pipeline.fit(train_titanic_data)

In [20]:
results = fit_model.transform(test_titanic_data)

In [21]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="Survived", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(results)
print("Accuracy = %g" % (accuracy))
print("Error rate = %g" % (1.0 - accuracy))